# 0. Package Installation and Disable Unecessary Logs

In [1]:
# -------------------- 0. Install and Import Dependencies --------------------
!pip install --upgrade transformers datasets peft accelerate

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging
import os

# Disable unnecessary logs and wandb
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)
os.environ["WANDB_DISABLED"] = "true"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 78.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia

2025-04-11 05:12:36.091373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744348356.293771      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744348356.350858      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 1. Load Model and Prepare Input Data

In [2]:
# -------------------- 1. Load Model and Tokenizer --------------------
# Path to the fully merged (standalone) best model
model_path = "/kaggle/input/comp34812_cw_fullmodel/pytorch/default/1/best_model_full"  # Update as needed

print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Loading model and tokenizer...


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

# 2. Load test.csv and Tokenization

In [3]:
# -------------------- 2. Load and Tokenize Test Data --------------------
print("Loading test.csv...")
df = pd.read_csv("/kaggle/input/comp34812-cw-c-demo-dataset/test_data/NLI/test.csv")  
texts = list(zip(df["premise"], df["hypothesis"]))

# Tokenize input sentence pairs
def encode(pairs):
    premises, hypotheses = zip(*pairs)
    return tokenizer(
        list(premises),
        list(hypotheses),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

encodings = encode(texts)

# Dataset class for tokenized test data
class NLIDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.encodings.items()}

    def __len__(self):
        return len(self.encodings["input_ids"])

test_dataset = NLIDataset(encodings)

Loading test.csv...


# 3. Batch Inference

In [4]:
# -------------------- 3. Run Inference --------------------
# Use efficient batch inference
eval_args = TrainingArguments(
    output_dir="/kaggle/working/tmp",  # Dummy output dir
    per_device_eval_batch_size=64,
)

trainer = Trainer(model=model, args=eval_args)

print("Running inference on test data...")
outputs = trainer.predict(test_dataset)
preds = outputs.predictions.argmax(-1)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Running inference on test data...


# 4. Save test.csv's Prediction

In [5]:
# -------------------- 4. Save Predictions --------------------
# Save predictions to required format: Group_n_category.csv
output_df = pd.DataFrame({"prediction": preds})
output_df.to_csv("/kaggle/working/Group_48_C.csv", index=False)
print("Saved predictions to Group_48_C.csv")

Saved predictions to Group_48_C.csv
